# Deployment

We take the model from Session_4:

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [8]:
df = pd.read_csv('churn_data.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
y_test = df_test['churn'].values

numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]


def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

C = 1.0
n_splits = 5

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
print(f'auc_score = {auc}')

C=1.0 0.840 +- 0.008
auc_score = 0.8572386167896259


### Save model

![](./pic/1.png)

- we have model in notebook
- we want to save it into file
- then we want to load it into web-service
- then we want use model on web-service which will interact with other services etc

![](./pic/2.png)

- we have model
- we put it on web service (**FLASK** (framework for web-services creation in Python) for example)
- we want to isolate dependencies on that service that they do not interfere with other services on our machine (environment for Python dependencies (**Pipenv**))
- isolate system dependencies (**Docker**)
- deploy this on cloud (AWS - Elastic Beanstalk for example)

# Save Model

In [9]:
# we will use pickle to save model as python object:
import pickle
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [11]:
# we want to write (w) into binary (b) file
f_out = open('output_file', 'wb')
pickle.dump(model, f_out)
f_out.close()

but now we saved only Model, however in Predict function, we also had DictVectorizer required, otherwise we would not be able to properly transform data from users

In [12]:
# that is why we will write Tuple (dv, model):
f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out) # dv added
f_out.close()

In [13]:
# it is easy to forget to close the file, so it is better to use with:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

# Load model

In [3]:
import pickle

In [4]:
input_file = 'model_C=1.0.bin'

In [5]:
with open(input_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
model

LogisticRegression(max_iter=1000)

In [14]:
#new customer:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [15]:
X = dv.transform([customer])

model.predict_proba(X)[0, 1]

0.6363584152721566

So we can put these pipeline into .py scripts (train.py and predict.py)  
and use them later

# Web services

we want to put our model in some kind of Churn Service (Web-service)

- Web-service - a way of communicating between 2 devices via Network

We want to use our predict script as Web-service - create API for our model

![](./pic/3.png)

When our Service is running, we may try to pretend that we here is the Marketing Service sending Request to our Churn Service:

when we try to access our predict Sevice from browser, we get an Error: "Method Not Allowed", since browser sends the Get request but, the only possible method we coded is POST,

therefore we need to POST using Pyhton library "requests";

In [3]:
# requests
import requests
# url is the one that we used creating Predict Web-service
url = 'http://localhost:9696/predict'
# customer in json format:
customer = {
    "gender": "female",
    "seniorcitizen": 0,
    "partner": "yes",
    "dependents": "no",
    "phoneservice": "no",
    "multiplelines": "no_phone_service",
    "internetservice": "dsl",
    "onlinesecurity": "no",
    "onlinebackup": "yes",
    "deviceprotection": "no",
    "techsupport": "no",
    "streamingtv": "no",
    "streamingmovies": "no",
    "contract": "month-to-month",
    "paperlessbilling": "yes",
    "paymentmethod": "electronic_check",
    "tenure": 1,
    "monthlycharges": 29.85,
    "totalcharges": 29.85
}

# making POST request to the Predict Web Service:
requests.post(url, json=customer)

<Response [500]>

We have got Error 500, in Terminal we can see:  
"TypeError: Object of type bool_ is not JSON serializable"

"bool_" is something that comes from numpy (response) from model, so we can convert bool response of our model into bool datatype explicitly

In [4]:
requests.post(url, json=customer)

<Response [500]>

now the response is 200, which means OK

In [8]:
# get response content:
response = requests.post(url, json=customer).json()
response

{'churn': True, 'churn_probability': 0.6363584152721566}

In [9]:
# Marketing service app:
if response['churn'] == True:
    print(f'sending promo email to customer: 999')

sending promo email to customer: 999


WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.

the reason for this warning is that we using plain (simple) Flask,  
to eliminate this, we need to use WSGI server instead
- WSGI - **Web Server Gateway Interface**. It is a specification that describes how a web server communicates with web applications, and how web applications can be chained together to process one request.



we will use **gunicorn** (pip isnstall gunicorn)

then we need to tell gunicorn where our Flask app is:  
*gunicorn --bind 0.0.0.0:9696 predict:app*  
(The module fctnl is not available on Windows systems), so we can either install ubuntu terminal or use "Waiters":  
*pip install waitress* 

  
  

waitress-serve --listen=0.0.0.0:9696 predict:app

In [8]:
# get response content:
response = requests.post(url, json=customer).json()
response

{'churn': True, 'churn_probability': 0.6363584152721566}

# Python virtual environment: Pipenv

if one project uses sklearn of one version and another project uses different version, they need to be able to leave on the same computer

## Intro

When we want to install some library (for example sklearn) we type: *pip install sklearn*

in that type computer looks at the $PATH direction to find what **pip** we are talking about, there then it is able to find path to the pip file, which then is running and requesting pypi.org for the latest or some specific version of lybrary

now for example, let us say that we need sklearn 0.24.2 for our churn prediction project, however some other project (for example Lead Scoring Service) requires other version of sklearn 
- Lead Scoring - the evaluation that potential client will become our client in future  


therefore, if we had these 2 projects on the same computer: we may have situation when some changes in later version of library influences the work of 1st project  
so we need to find a way to keep them separated

the solution is to use different virtual environments

![](./pic/4.png)

so these projects use different pythons, with their own packages

### Virtual environments

there are multiple ways of managing virtual environments:
- classical python build in /venv
- conda - what anaconda uses
- pipenv - the one we will use but there are no big difference, the reason we will use it is officially recommended package management from python community
- poetry